In [ ]:
!poetry add langchain_ollama langchain_community langchain_chroma pymupdf langchain_openai python-dotenv

The following packages are already present in the pyproject.toml and will be skipped:

  - langchain_ollama
  - langchain_community
  - langchain_chroma
  - pymupdf

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Using version ^0.3.8 for langchain-openai

Updating dependencies
Resolving dependencies...

Package operations: 5 installs, 0 updates, 0 removals

  - Installing jiter (0.9.0)
  - Installing regex (2024.11.6)
  - Installing openai (1.66.3)
  - Installing tiktoken (0.9.0)
  - Installing langchain-openai (0.3.8)

Writing lock file


In [7]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, load_prompt
from langchain_core.runnables import RunnablePassthrough
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
retriever_setting = [
    {"k" : 10}, #opt_1
    {"k" : 10, "fecth_k" : 12}, #opt2
    {"k" : 10, "fecth_k" : 12, "lambda_mult" : 0.7}, #opt3

]

In [5]:
docs_path = "./docs/T-Series Subsea_OpMan_12.pdf"
emdedding_model = "bge-m3:latest"
llm_model = "gemma3:4b"
prompt_path = "./prompt/manual.yaml"
openai_llm = "gpt-4o-mini"
openai_embedding = "text-embedding-3-large"

In [9]:
loader = PyMuPDFLoader(docs_path)

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 800, 
    chunk_overlap = 150
    )


document = loader.load()
splitted_document = splitter.split_documents(document)

In [24]:
llm = OllamaLLM(model=llm_model)

embedding = OllamaEmbeddings(model=emdedding_model)

prompt = load_prompt(prompt_path)

vector_store = Chroma(
    collection_name="mbes_db",
    embedding_function=embedding,
    persist_directory="./database"
)

basic_mmr_retriever = vector_store.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k" : 10,}
    )

In [25]:
llm = ChatOpenAI(model=openai_llm)

embedding = OllamaEmbeddings(model=emdedding_model)

prompt = load_prompt(prompt_path)

vector_store = Chroma(
    collection_name="mbes_db",
    embedding_function=embedding,
    persist_directory="./database"
)

basic_mmr_retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k" : 50,}
    )

In [ ]:
vector_store.from_documents(
    documents=splitted_document,
    embedding=embedding,
    collection_name="mbes_db",
    persist_directory="./database"
)

In [11]:
print(prompt)

input_variables=['equipment'] input_types={} partial_variables={} template='\n**[Goal]**  \nCreate a structured manual specifically focusing on the software initialization and configuration procedures for {equipment}. The manual should guide users clearly from the initial software launch through necessary settings adjustments, and include situational setting instructions based solely on software operations.\n\n**[Constraints]**  \n- Only information provided by RAG documents is allowed.  \n  (Do not add any data not explicitly mentioned in the RAG.)\n- Structure must follow a clear, logical, and sequential manual format.\n- Hardware operation or non-software related details must not be included.\n- Prepare the structure and content first in English, then translate the final output into clear, understandable Korean for easy comprehension by Korean users.\n\n**[Required Manual Structure]**\n\n1. **Software Initialization Procedure**\n  - Starting the software (step-by-step launch instruc

In [26]:
chain = (
    {
        "context" : basic_mmr_retriever,
        "equipment" : RunnablePassthrough() 
        } |
    prompt |
    llm
)

In [27]:
res = chain.invoke("T-50s")

In [28]:
print(res.content)

### SeaBat T-Series Subsea Software Setup Manual

1. Execute the software "Sonarsis" for the T-50s equipment.
2. Click the [Setup] menu from the main screen.
3. Press the [New Config] button to add a new configuration.
4. Click the [New Project] button to create a new project.
5. Enter your desired project name into the [Project Name] field, then click the [Confirm] button.
6. In the [Preferences] menu, configure the settings as follows:
   - Navigate to the [Network] tab, enter the appropriate IP address and DNS settings.
   - Open the [Config] tab and input GPS coordinates and configure motion sensor settings based on your project requirements.
7. After all required settings are configured, click the [Apply] button to apply changes and then click the [Save] button to store these settings.
8. Once all configurations are complete, click [Exit] to close the settings menu. 

Always follow these steps to ensure successful configuration of the SeaBat T-Series Subsea system software.
